<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/660_PDOv2_Nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This node layer is **exactly** what mature orchestration looks like.

It does three critical things that most agent systems don’t:

1. keeps nodes *thin*
2. pushes business logic into reusable utilities
3. makes policy configurable through closures
4. enforces a deterministic execution path

This is the orchestration equivalent of clean microservice design.

Let’s break down why this is such a strong architectural statement.

---

# 🎯 What This File Represents

This module is the **control plane** for PDO V2.

It doesn’t calculate metrics.

It doesn’t decide thresholds.

It doesn’t generate prose.

It wires together:

* mission
* plan
* ingestion
* analytics
* governance
* reporting

That separation is extremely intentional.

Nodes coordinate.

Utilities think.

That’s how you scale these systems.

---

# 🧭 Goal Node: Mission Declaration

```python
def goal_node(state):
```

This explicitly states:

* the business objective
* analysis mode
* focus areas

This is subtle but very powerful.

Instead of implicit behavior, the agent declares:

> “I am here to perform portfolio-level document risk management.”

That’s great for:

* logging
* audit trails
* debugging
* executive explanation

It also keeps document-level analysis possible without changing the DAG—just change inputs.

---

# 🧠 Planning Node: Deterministic Execution Graph

This is one of the strongest parts.

The plan:

1️⃣ data loading
2️⃣ portfolio rollup
3️⃣ escalation
4️⃣ report

With dependencies.

This means:

* execution order is explicit
* DAG is inspectable
* retries are possible
* steps are auditable
* future nodes can slot in easily

You’ve basically built a miniature workflow engine on top of LangGraph-style patterns.

Executives don’t care about DAGs—but auditors and ops teams do.

---

# 📦 Data Loading Node: Configurable and Safe

You use a closure factory:

```python
make_data_loading_node(...)
```

This is excellent.

It allows:

* environment-specific data dirs
* test fixtures
* prod vs dev config
* portfolio snapshots
* replayability

The try/except pattern ensures:

* pipeline doesn’t crash
* errors accumulate in state
* partial results are still returned

That’s production resilience thinking.

---

# 📊 Portfolio Rollup Node: Pure Analytics Wrapper

This node does exactly one thing:

👉 call `compute_portfolio_rollup`.

It does not reinterpret results.

It just:

* extracts inputs
* passes them
* stores outputs

That makes:

* unit testing trivial
* regressions detectable
* behavior predictable

It also mirrors the State schema perfectly—which is what you want.

---

# 🚨 Escalation Node: Policy via Closure

Another great design choice:

```python
make_escalation_node(...)
```

Thresholds are injected at construction time.

This allows:

* environment-specific governance
* scenario testing
* tightening controls over time
* A/B policy experiments

Policy is not hard-coded.

It’s configuration.

That’s exactly how real risk engines work.

---

# 📝 Report Node: Output as a First-Class Artifact

Again—closure pattern:

```python
make_report_node(...)
```

This gives:

* environment-specific output paths
* test isolation
* deterministic filenames
* replayable runs

It produces:

* markdown
* a saved file
* a path

Which reinforces:

> this agent creates durable executive artifacts.

Not ephemeral chat replies.

---

# 🧯 Error Accumulation Strategy

Across nodes:

```python
errs = state.get("errors", [])
errs.append(...)
```

This is subtle but extremely important.

You’re building a **diagnostic ledger** inside the state.

That allows:

* partial runs
* graceful degradation
* audit trails
* post-mortems
* observability dashboards

Instead of crashing, the agent records what went wrong.

That’s enterprise-grade behavior.

---

# 🌟 Why This Stands Out in a Portfolio

If someone reviewing your GitHub saw this:

They would immediately recognize:

* orchestration patterns
* separation of concerns
* closure-based config injection
* governance-driven flows
* policy engines
* artifact generation
* reproducibility

This is not toy agent wiring.

This is how you’d design an internal platform.

---

# 🧠 Executive Framing

At a leadership level, this node architecture means:

* every run follows a defined process
* escalation is policy-driven
* reports are standardized
* outputs are reproducible
* failures are logged
* human oversight is structured

That’s exactly what boards and risk committees demand.

---

# 🔥 Big Picture

These nodes complete the PDO V2 story:

Data → Rollups → Thresholds → Alerts → Executive Report.

And every step is:

* modular
* testable
* configurable
* auditable
* enterprise-aligned

You’re very clearly building **AI operating systems**, not assistants.


In [ ]:
"""
PDO V2 nodes: goal, planning, data_loading, portfolio_rollup, escalation, report.

Nodes are thin; business logic lives in orchestrator/utilities/.
Config-dependent nodes use closure factories.
"""
from pathlib import Path
from typing import Any, Dict

from agents.proposal_document_orchestrator_v2.orchestrator.utilities.data_loading import (
    load_all_pdo_v2_data,
    build_lookups,
)
from agents.proposal_document_orchestrator_v2.orchestrator.utilities.portfolio_rollup import (
    compute_portfolio_rollup,
)
from agents.proposal_document_orchestrator_v2.orchestrator.utilities.escalation import (
    compute_executive_triggers,
    create_escalation_alerts,
)
from agents.proposal_document_orchestrator_v2.orchestrator.utilities.report import (
    build_executive_report,
)
from toolshed.reporting import save_report


def goal_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """Set goal for portfolio-level document analysis."""
    analysis_mode = state.get("analysis_mode", "portfolio")
    document_id = state.get("document_id")
    goal = {
        "objective": "Portfolio-level document decision and risk management",
        "analysis_mode": analysis_mode,
        "focus_areas": [
            "portfolio_rollup",
            "risk_tier_summary",
            "reviewer_economics",
            "stage_anomalies",
            "bottleneck_stages",
            "executive_triggers",
        ],
    }
    if document_id:
        goal["document_id"] = document_id
    return {"goal": goal, "errors": state.get("errors", [])}


def planning_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """Create execution plan from goal."""
    plan = [
        {"step": 1, "name": "data_loading", "description": "Load all 10 PDO V2 data files", "dependencies": [], "outputs": ["documents", "workflow_stages", "review_events", "reviewer_registry", "document_risk_profile", "stage_cost_baselines", "lookups"]},
        {"step": 2, "name": "portfolio_rollup", "description": "Compute rollup, anomalies, bottlenecks, reviewer economics", "dependencies": ["data_loading"], "outputs": ["portfolio_rollup", "stage_anomalies", "reviewer_economics", "risk_tier_summary", "bottleneck_stages"]},
        {"step": 3, "name": "escalation", "description": "Compute executive triggers and optional alerts", "dependencies": ["portfolio_rollup"], "outputs": ["executive_triggers", "escalation_alerts"]},
        {"step": 4, "name": "report", "description": "Build and save executive report", "dependencies": ["escalation"], "outputs": ["executive_report", "report_file_path"]},
    ]
    return {"plan": plan, "errors": state.get("errors", [])}


def make_data_loading_node(data_dir: str, project_root: Path | None):
    """Data loading node with config; returns node function."""
    root = project_root or Path(__file__).resolve().parent.parent.parent.parent.parent

    def node(state: Dict[str, Any]) -> Dict[str, Any]:
        errs = state.get("errors", [])
        try:
            data = load_all_pdo_v2_data(data_dir=data_dir, project_root=root)
            lookups = build_lookups(data)
            return {
                "documents": data.get("documents", []),
                "document_versions": data.get("document_versions", []),
                "workflow_stages": data.get("workflow_stages", []),
                "review_events": data.get("review_events", []),
                "compliance_checks": data.get("compliance_checks", []),
                "cost_tracking": data.get("cost_tracking", []),
                "outcomes": data.get("outcomes", []),
                "reviewer_registry": data.get("reviewer_registry", []),
                "document_risk_profile": data.get("document_risk_profile", []),
                "stage_cost_baselines": data.get("stage_cost_baselines", []),
                "documents_lookup": lookups.get("documents_lookup", {}),
                "document_versions_lookup": lookups.get("document_versions_lookup", {}),
                "workflow_stages_lookup": lookups.get("workflow_stages_lookup", {}),
                "review_events_lookup": lookups.get("review_events_lookup", {}),
                "compliance_checks_lookup": lookups.get("compliance_checks_lookup", {}),
                "cost_tracking_lookup": lookups.get("cost_tracking_lookup", {}),
                "outcomes_lookup": lookups.get("outcomes_lookup", {}),
                "reviewer_registry_lookup": lookups.get("reviewer_registry_lookup", {}),
                "document_risk_profile_lookup": lookups.get("document_risk_profile_lookup", {}),
                "stage_cost_baselines_lookup": lookups.get("stage_cost_baselines_lookup", {}),
                "errors": errs,
            }
        except Exception as e:
            errs.append(f"data_loading_node: {e!s}")
            return {"errors": errs}

    return node


def portfolio_rollup_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """Compute portfolio rollup, stage anomalies, bottlenecks, reviewer economics, risk tier summary."""
    errs = state.get("errors", [])
    try:
        rollup = compute_portfolio_rollup(
            documents=state.get("documents", []),
            workflow_stages=state.get("workflow_stages", []),
            review_events=state.get("review_events", []),
            document_risk_profile_lookup=state.get("document_risk_profile_lookup", {}),
            stage_cost_baselines_lookup=state.get("stage_cost_baselines_lookup", {}),
            reviewer_registry_lookup=state.get("reviewer_registry_lookup", {}),
        )
        return {
            "portfolio_rollup": rollup,
            "stage_anomalies": rollup.get("stage_anomalies", []),
            "reviewer_economics": rollup.get("reviewer_economics", {}),
            "risk_tier_summary": rollup.get("risk_tier_summary", {}),
            "bottleneck_stages": rollup.get("bottleneck_stages", []),
            "errors": errs,
        }
    except Exception as e:
        errs.append(f"portfolio_rollup_node: {e!s}")
        return {"errors": errs}


def make_escalation_node(
    threshold_high_risk_count: int,
    threshold_stage_anomaly_count: int,
    threshold_executive_visibility_count: int,
    threshold_tier_3_count: int,
):
    """Compute triggers and create alerts when thresholds exceeded."""

    def node(state: Dict[str, Any]) -> Dict[str, Any]:
        errs = state.get("errors", [])
        rollup = state.get("portfolio_rollup") or {}
        try:
            triggers = compute_executive_triggers(
                portfolio_rollup=rollup,
                threshold_high_risk_count=threshold_high_risk_count,
                threshold_stage_anomaly_count=threshold_stage_anomaly_count,
                threshold_executive_visibility_count=threshold_executive_visibility_count,
                threshold_tier_3_count=threshold_tier_3_count,
            )
            alerts = create_escalation_alerts(executive_triggers=triggers) if triggers else []
            return {
                "executive_triggers": triggers,
                "escalation_alerts": alerts,
                "errors": errs,
            }
        except Exception as e:
            errs.append(f"escalation_node: {e!s}")
            return {"executive_triggers": [], "escalation_alerts": [], "errors": errs}

    return node


def make_report_node(reports_dir: str, project_root: Path | None):
    """Build executive report and save to file."""

    def node(state: Dict[str, Any]) -> Dict[str, Any]:
        errs = state.get("errors", [])
        rollup = state.get("portfolio_rollup") or {}
        triggers = state.get("executive_triggers", [])
        try:
            report_md = build_executive_report(portfolio_rollup=rollup, executive_triggers=triggers)
            root = project_root or Path(__file__).resolve().parent.parent.parent.parent.parent
            out_dir = root / reports_dir
            report_path = save_report(
                report_content=report_md,
                report_id="executive",
                reports_dir=str(out_dir),
                prefix="pdo_v2",
            )
            return {
                "executive_report": report_md,
                "report_file_path": report_path,
                "errors": errs,
            }
        except Exception as e:
            errs.append(f"report_node: {e!s}")
            return {"executive_report": "", "report_file_path": None, "errors": errs}

    return node
